In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# auto differentiation library
import torch
from torch.autograd import grad, Function
import torch.nn.functional as F  # usual functions
import torch.nn as nn  # for defining Neural Networks
import torch.optim as optim

# notebook specific library for displays
from IPython.display import display, Markdown

# Pytorch's Tensors

Pytorch contains a class `Tensor` which is a wrapper for array, matrices, tensors... But numpy is already able to handle usual linear algebra operation and coordinate-wise operations. Pytorch brings to the table __auto-differentiation__ and __GPU operations__.
That means a Tensor contains everything necessary to backpropagate gradients through the operation which created it.

In [ ]:
# defining a Tensor from a numpy array
x = np.array([3, 1, 4, 1, 5, 9, 2])
x = torch.Tensor(x)
print("Tensor type `x.type()` gives more insight on the data structure of tensor coordinates: {}".format(x.type()))
print("The tensor's shape, alternatively `x.size()` or `x.shape`: {}".format(x.shape))
print(x)
print("You can get a numpy array back with `x.numpy()`: {}".format(x.numpy()))

In [ ]:
# linear algebra operation
A = np.random.randn(3, x.shape[0])
A = torch.Tensor(A)

print("`torch.mv` for matrix-vector multiplication: {}".format(torch.mv(A, x)))
print("Multiplication `*` is reserved for coordinate wise multiplication.")
print("In Python, matrix multiplication is done using `@`: {}".format(A @ x))

# Pytorch's Gradient Computation

Let's say $y = f(x) \in \mathbb R$ where $f$ is a known function and $x$ is a tensor. We know how to compute the gradient of $f$ at point $x$: for Pytorch, "knowing" a function means knowing both how to compute its value at a point $x \mapsto f(x)$, but also knowing how to compute its gradient $x \mapsto \nabla f(x)$.

When computing $y$, Pytorch records that it was obtained from calling the function $f$ at point $x$. Here is an example of this computation using only one variable $y = \| x \|_2$:

In [ ]:
x = torch.randn(4, requires_grad=True)
y = torch.norm(x, p='fro')
display(Markdown(r"Pytorch knows how y was defined, it records how to compute compute gradients of y: $\nabla \|\,.\|_2$ is stored as '{}'".format(y.grad_fn)))

### Computing a gradient with Pytorch

In this context, $y$ represents an error (or "loss" as it is often called in machine learning) that we want to minimize with a gradient descent. 

If the user wants to compute the gradient of $y$ with respect to $x$:
- The gradient we want to compute is the gradient of $y$ with respect to other variables: we know that $y = f(x)$ so we want to compute $\nabla f(x)$.
- $f$ is a "known" function, so pytorch has access to $\nabla f$, and it has recorded the value of $x$ as well: it simply needs to compute $\nabla f(x)$.

This computation is done when calling `torch.grad` which was imported as `grad`. It needs two arguments:
- The loss whose gradient we want to compute $y$
- The list of variables with respect to which we want to compute the gradient: if $\mathcal y = f(x, z)$, and we want the gradient with respect to both $x = (x_1, \dots, x_n)$ and $z = (z_1, \dots, z_m)$, then the output will be
$$
    \nabla f(x, z) = \left(
        \frac{\partial \mathcal L}{\partial x_1}, \dots, \frac{\partial \mathcal L}{\partial x_n},
        \frac{\partial \mathcal L}{\partial z_1}, \dots, \frac{\partial \mathcal L}{\partial z_m}
    \right) = \left( \nabla_x f(x, z) \ | \ \nabla_z f(x, z) \right)
$$

Here is what it looks like with the previous example $y = \| x \|_2$:

In [ ]:
grad_y = grad(y, [x])
print("grad returns a list of gradients refering to the list of arguments [x]: {}".format(grad_y))
grad_y_x = grad_y[0]  # index 0 because x is at index 0 in the list [x]
display(grad_y_x)

#### Exercise 1 - Sanity check: compute the gradient $\nabla_x y$ by hand and compare your result to `grad_y_x`

In [ ]:
# your code goes here

### Other differential operators:

Know that the `torch.autograd` package contains automatic computations for other differential operators. For example, the function `jacobian` extends `grad` to non-scalar inputs and the function `hessian` computes second order derivatives.

# First mention of a neural network

Let's present an abstract description of neural networks through a basic example: recognizing photos of cats and dogs. 
- $x \in \mathbb R^d$ contains a data sample: in this example $x$ would be the image, but in other contexts it could be sounds, medical records, GPS informations, internet history, DNA information, etc... or even a combination of multiple of those.
- For a binary classification task, $y \in \{0, 1\}$ represents the prediction that we make for that sample e.g. if the image in $x$ is more likely to be a cat $0$ or a dog $1$ ?

Often, we will assume that there exists an _oracle_ function $f:\, \mathbb R^d \mapsto \{0, 1\}$ such that $f(x)$ is always the correct answer, or at least the best$^*$ answer that can be given.

*: here, the "best" is defined in relation to an error that must be defined as part of the mathematical formulation of the problem we are confronted to.

### An abstract neural network

In general, neural networks are a class of parametrized functions $f_\theta$ mapping an input to a prediction: $y = f_\theta(x)$. Here, $\theta \in \mathbb R^p$ is the set of parameters used by the network.

The goal is to make good prediction, _i.e._ to estimate the class of any probable data sample $x$: $\tilde y = f_\theta(x)$ approximates $y = f(x)$.

### Machine Learning technical jargon
In supervized learning, we fix the parameters $\theta$ with what refer to as training, using a dataset.
- A dataset is a set of pairings between data samples $x^{(1)} \in \mathbb R^d,\, \dots,\, x^{n} \in \mathbb R^d$, and the corresponding correct answer:
    - for example, humans are very good at recognizing cats from dogs on images so the network can be trained to match human performance
    - in other contexts, the correct answer can be an objective truth: did the meteorological $x^{(i)}$ result in rain one hour later ? did the patient carrying DNA $x^{(i)}$ develop a specific cancer before the age of 50 ? Did the internet user with the internet history $x^{(i)}$ buy the product advertised ?
- We can rewrite $f_\theta(x) = f(x, \theta)$. $f$ is a function defined by the user, so its gradient is known: we don't care much about $\nabla_x f$, but $nabla_\theta f$ allows us to optimise $\theta$ through a gradient decent. Training is just technical jargon for this gradient descent.

Note: to have well defined gradients, we can't use $\tilde y = f_\theta(x) \in \{0, 1\}$ and relax it to $\tilde y \in [0, 1]$ or even $\tilde y \in \mathbb R$.

### Computing $\nabla_\theta f$
In general, $f_\theta$ can be very complicated and include several milions of parameters: deriving its gradient by hand can be tidious if not physically impossible.

However, $f_\theta$ is often defined as a cascade of simple functions: linear functions $A_k: \mathbb R^{d_k} \mapsto \mathbb R^{d_{k+1}}$ and non-linear activations $\rho_k: \mathbb R \mapsto \mathbb R$ applied to each coordinate independantly. The parameters here are the matrices $A_k$ which are regrouped into $\theta = \left( A_1 | \dots | A_K \right)$.

This structure allows us to compute all gradients
$$
    \nabla_\theta f(x, \theta) = \left(\ \nabla_{A_1} f(x, \theta) \,| \dots \,|\, \nabla_{A_K} f(x, \theta) \ \right)
$$
using the chain-rule. The resulting algorithm is called backpropagation, we will study it next time.

# Gradient Descent in Pytorch

Let $A \in \mathcal M_{p, n}(\mathbb R)$ and $y \in \mathbb R^p$. We want to find the pseudo inverse $x = A^{<-1>}y \in \mathbb R^n$ of $y$. This is typically done by defining
$$
    \tilde x = argmin_{x \in \mathbb R^n} \| Ax - y \|_2^2
$$

In [ ]:
p, n = 3, 4
A = torch.randn(p, n)
y = torch.randn(p)

#### Exercise 2: Using a random initialization and Pytorch's SGD (stochastic gradient descent): https://pytorch.org/docs/stable/optim.html?highlight=sgd#torch.optim.SGD , compute $\tilde x$. Plot the evolution of $\| Ax - y \|_2^2$ during the optimization.

# Defining functions in Pytorch

Pytorch already knows many elementary functions such as sums, sine functions, norms, etc... Other functions can be obtained through the chain rule.

#### Exercise 3: Define the function $f(x, A, b) = \max(0, Ax + b)$ where $x \in \mathbb R^{d_0}$, $A \in \mathcal M_{d_0, d_1}(\mathbb R)$ and $b \in \mathbb R^{d_1}$.

Note: $x \mapsto \max(0, x)$ is a widely used operation in deep learning, commonly refered to as ReLU for _Rectified Linear Unit_

In [ ]:
def f(x, A, b):
    """Compute the result of a fully connected layer y = max(0, Ax+b)."""

    # your code goes here
    pass

In [ ]:
d0, d1 = 4, 3

x = torch.randn(d0, requires_grad=True)
A = torch.randn(d1, d0, requires_grad=True)
b = torch.randn(d1)

#### Exercise 4: Compute $\|f(x, A, b)\|_2$ and its gradient with respect to $A$ and $b$ using the function `grad` as before. How could Pytorch know how to compute those gradients ?

In [ ]:
# your code goes here

### Defining new functions

If the function we want to use can not be defined using basic Pytorch functions (typically because one step is unstable or has unstable gradient computation), we must define it ourselves.

Pytorch's autograd package provides the class `Function` for this exact purpose:
    https://pytorch.org/docs/stable/autograd.html?highlight=function#torch.autograd.Function

Recall that in Pytorch, "knowing" a function a function means knowing how to compute it, as well as knowing how to compute its gradient.

#### Exercise 5: Define the inverse matrix function

In [ ]:
class InverseMatrix(Function):
    """A Function class for computing the inverse matrix, and its gradient."""
    
    @staticmethod
    def forward(ctx, A):
        """Compute the inverse of matrix A"""
        pass
    
    @staticmethod
    def backward(ctx, grad_output):
        """Compute the gradient of the loss with respect to A.
        
        grad_ouput contains the gradient of the loss with respect to A^-1
        """
        pass